# Implementation

## Imports

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os

In [2]:
plt.style.use('dark_background')

In [3]:
import sklearn
import keras
import tensorflow as tf

In [4]:
from keras.applications import InceptionV3

In [5]:
import cv2

In [6]:
path = os.getcwd()
path = path[:-4]
data_dir = path + "/data/lfw-deepfunneled"

In [7]:
people = pd.read_csv(path + "/data/people.csv")
people = pd.read_csv(path + "/data/people.csv")
names = [item for item in people["name"].tolist() if item in os.listdir(data_dir)]
people = people[people["name"].isin(names)]
N = 6
people = people[people.images > N]
k = len(people)
print(f"k = {k} classes")

k = 256 classes


In [8]:
inceptionv3 = InceptionV3()
model = InceptionV3(include_top=False, classes=k, pooling='avg', weights='imagenet')
model = keras.Sequential(layers=(model, keras.layers.Dense(k, activation='softmax')))

## Data Preparation

In [9]:
people = people.rename(columns={'images': 'num_images'})

In [10]:
people = people.set_index('name')

In [11]:
image_col = []
for person in people.iterrows():
    person = person[1]
    image_list = []
    for i in range(int(person["num_images"])):
        image_path = data_dir + '/' + person.name + '/' + person.name + '_' +  "{:04d}".format(i+1) + '.jpg'
        image = cv2.imread(image_path)
        image = cv2.resize(image, (299, 299))
        #image = tf.expand_dims(image, axis=0)
        image_list.append(image)
    image_col.append(image_list)

In [12]:
people["images"] = image_col

In [13]:
# num_images == len(images) for each person
assert ([len(x) for x in people["images"]] == people["num_images"].values).all()

In [14]:
dataset = [] # element: (image, person, train or test)
for person in people.iterrows():
    person = person[1]
    image_list = person["images"]
    num_images = len(image_list)
    i_list = np.random.choice(num_images, N)
    test_i = i_list[np.random.randint(0, N)]
    for i in i_list:
        image = image_list[i]
        element = [image, person.name, "test" if i==test_i else "train"]
        dataset.append(element)
dataset = pd.DataFrame(dataset, columns=["image", "person", "split"])

In [15]:
dataset = pd.get_dummies(dataset, columns=['person'])

In [16]:
dataset.head()

,image,split,person_Abdullah_Gul,person_Adrien_Brody,person_Al_Gore,person_Al_Sharpton,person_Alejandro_Toledo,person_Ali_Naimi,person_Alvaro_Uribe,person_Amelia_Vega,...,person_Walter_Mondale,person_Wen_Jiabao,person_William_Donaldson,person_William_Ford_Jr,person_Winona_Ryder,person_Yao_Ming,person_Yashwant_Sinha,person_Yasser_Arafat,person_Yoriko_Kawaguchi,person_Zhu_Rongji
0,"[[[129, 163, 176], [129, 163, 176], [129, 163,...",train,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,"[[[0, 0, 0], [0, 0, 0], [1, 0, 0], [2, 0, 0], ...",train,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,"[[[10, 10, 0], [5, 5, 0], [4, 4, 0], [5, 5, 0]...",train,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",train,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,"[[[0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], ...",test,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


## Model Training on LFW Dataset

<img src="fig2.png" alt="drawing" width="600"/><br>
1. Train model using N-1 images of each person.
2. Use remaining image as test dataset, get results.

In [17]:
X_train = dataset[dataset["split"] == "train"]["image"].tolist()
X_train = np.asarray(X_train).astype('float32')
X_test = dataset[dataset["split"] == "test"]["image"].tolist()
X_test = np.asarray(X_test).astype('float32')

In [18]:
y_train = dataset[dataset["split"] == "train"].drop(['image', 'split'], axis=1)
y_test = dataset[dataset["split"] == "test"].drop(['image', 'split'], axis=1)

In [19]:
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-4), metrics=['accuracy'])

In [20]:
model.fit(X_train, y_train, shuffle=True, epochs=10)

Epoch 1/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 264s 7s/step - accuracy: 0.0469 - loss: 5.5653
Epoch 2/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 273s 7s/step - accuracy: 0.7788 - loss: 3.7130
Epoch 3/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 272s 7s/step - accuracy: 0.9776 - loss: 2.3028
Epoch 4/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 274s 7s/step - accuracy: 0.9920 - loss: 1.3061
Epoch 5/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 278s 7s/step - accuracy: 1.0000 - loss: 0.6519
Epoch 6/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 294s 8s/step - accuracy: 1.0000 - loss: 0.3053
Epoch 7/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 293s 8s/step - accuracy: 1.0000 - loss: 0.1696
Epoch 8/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 293s 8s/step - accuracy: 1.0000 - loss: 0.1019
Epoch 9/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 293s 8s/step - accuracy: 1.0000 - loss: 0.0750
Epoch 10/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 291s 8s/step - accuracy: 1.0000 - loss: 0.0583


In [21]:
p = model(tf.expand_dims(X_train[0], axis=0))

In [22]:
np.argmax(p)

14

In [23]:
np.argmax(y_train.iloc[0])

14

In [24]:
p = model(tf.expand_dims(X_test[0], axis=0))

In [25]:
np.argmax(p)

14

In [26]:
np.argmax(y_test.iloc[0])

14

In [27]:
model.evaluate(X_test, y_test)

12/12 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.2974 - loss: 4.0167


[3.971081495285034, 0.3125]

## Conclusion
Model tends to fit closely to train data. Perhaps with more train data (and thus more variations put in model such as spatial orientation) can good generalization be truly achieved.